In [2]:
# https://www.kaggle.com/datasets/christofferms/pokemon-with-stats-and-image/data

import pandas as pd

data = pd.read_csv("pokedex.csv")

data

,Image,Index,Name,Type 1,Type 2,Total,HP,Attack,Defense,SP. Atk.,SP. Def,Speed
0,images/1.png,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,images/2.png,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,images/3.png,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,images/4.png,3,Venusaur Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,images/5.png,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...,...
1210,images/1211.png,1023,Iron Crown,Steel,Psychic,590,90,72,100,122,108,98
1211,images/1212.png,1024,Terapagos Normal Form,Normal,NaN,450,90,65,85,65,85,60
1212,images/1213.png,1024,Terapagos Terastal Form,Normal,NaN,600,95,95,110,105,110,85
1213,images/1214.png,1024,Terapagos Stellar Form,Normal,NaN,700,160,105,110,130,110,85


Snažil jsem se najít si vlastní dataset, který by mě zajímal a nakonec jsem nečekaně skončil u pokémonů, kde jsem si řekl, že by to mohl být zajímavý dataset na analýzu. Dataset obsahuje pokémony z 1-9 řady pokémonů.

Konkrétně u tohoto datasetu máme záznam o 1215 pokémonech s tím, že známe typy pokémonů, jejich podrobné statistiky, takže budeme schopni určit trend pokémonů, kterých je nejvíce, kteří jsou nejsilnější a poté utvořit shlukování, které pravděpodobně bude shlukovat podle typů pokémonů.

Aktuálně nejvíce 